<h1>Imports</h1>

In [1]:
# Importing necessary packages

import cv2
import mediapipe as mp
import math
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>Supplementary Functions</h1>

In [2]:
# Calculating angle between joints

def calculate_angle(point1, point2, point3):
    point1 = np.array(point1)
    point3 = np.array(point3)
    point2 = np.array(point2)
    
    radians = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angle = np.abs((radians * 180.0)/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [3]:
def calculate_distance(point1, point2):

    distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    
    return distance

<h2>Analyzing Trainer footage to set state thresholds based on Knee angles</h2>

In [4]:
state1_angle_threshold = [0, 0]
state2_angle_threshold = [0, 0]
state3_angle_threshold = [0, 0]

In [38]:
trainer_footage = "sample_videos/abdullah_potrait.mp4"
client_footage = './sample_videos/knees_pointing.mp4'

In [6]:
cap = cv2.VideoCapture(trainer_footage)

knee_angles = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                     
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                knee_angles.append(knee_angle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                knee_angles.append(knee_angle)
                        
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
        
# print min and max angles for elbow

print("Knee Angle: ", min(knee_angles), max(knee_angles))

# Set the threshold values for each state

state3_angle_threshold[0] = min(knee_angles)
state1_angle_threshold[1] = max(knee_angles)

mid_angle = round((state3_angle_threshold[0] + state1_angle_threshold[1])/2.0, 1)

state_mid_1 = round((state3_angle_threshold[0] + mid_angle)/2.0, 1)
state_mid_2 = round((mid_angle + state1_angle_threshold[1])/2.0, 1)

state3_angle_threshold[1] = state_mid_1
state2_angle_threshold[0] = state_mid_1
state2_angle_threshold[1] = state_mid_2
state1_angle_threshold[0] = state_mid_2

Knee Angle:  92.47710703699761 179.88701180012214


In [7]:
# print the threshold values for each state

print("State 1: ", state1_angle_threshold)
print("State 2: ", state2_angle_threshold)
print("State 3: ", state3_angle_threshold)

State 1:  [158.0, 179.88701180012214]
State 2:  [114.3, 158.0]
State 3:  [92.47710703699761, 114.3]


<p>Using knee angles will be better to transition through states.</p>
<p>This is because shoulders and elbows barely move throughout the movement</p>

<h2>Getting Angles from Trainer Footage</h2>

In [8]:
trainer_hip_angle_state1 = []
trainer_knee_angle_state1 = []
trainer_knee_feet_ratio_state1 = []

trainer_hip_angle_state2 = []
trainer_knee_angle_state2 = []
trainer_knee_feet_ratio_state2 = []

trainer_hip_angle_state3 = []
trainer_knee_angle_state3 = []
trainer_knee_feet_ratio_state3 = []

In [9]:
cap = cv2.VideoCapture(trainer_footage)

state = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            # Knee Feet Ratio Calculation
            
            knee_distance = calculate_distance(left_knee, right_knee)
            feet_distance = calculate_distance(left_ankle, right_ankle)
            
            trainer_knee_feet_ratio = knee_distance/feet_distance 
               
            # Knee Angle Calculation and state selection
            
            knee_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            
            
            if knee_angle >= state1_angle_threshold[0] and knee_angle <= state1_angle_threshold[1]:
                state = 1
            elif knee_angle >= state2_angle_threshold[0] and knee_angle <= state2_angle_threshold[1]:
                state = 2
            elif knee_angle >= state3_angle_threshold[0] and knee_angle <= state3_angle_threshold[1]:
                state = 3
            else:
                state = 0
                    
            if state == 1:
                trainer_knee_angle_state1.append(knee_angle.round(1))
                trainer_knee_feet_ratio_state1.append(trainer_knee_feet_ratio)  
            elif state == 2:
                trainer_knee_angle_state2.append(knee_angle.round(1))
                trainer_knee_feet_ratio_state2.append(trainer_knee_feet_ratio)
            elif state == 3:
                trainer_knee_angle_state3.append(knee_angle.round(1))
                trainer_knee_feet_ratio_state3.append(trainer_knee_feet_ratio)            
                
                
            # Hip Angle Calculation + Back Curvature
            
            hip_angle = 0
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                
            if state == 1:
                trainer_hip_angle_state1.append(hip_angle.round(1))
            elif state == 2:
                trainer_hip_angle_state2.append(hip_angle.round(1))
            elif state == 3:
                trainer_hip_angle_state3.append(hip_angle.round(1))
            
            
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [10]:
# print all trainer angle arrays

print("State 1:-\n")
print("Trainer Hip Angle State 1: ", trainer_hip_angle_state1)
print("Trainer Knee Angle State 1: ", trainer_knee_angle_state1)
print("Trainer Knee Feet Ratio State 1: ", trainer_knee_feet_ratio_state1)

print("\nState 2:-\n")
print("Trainer Hip Angle State 2: ", trainer_hip_angle_state2)
print("Trainer Knee Angle State 2: ", trainer_knee_angle_state2)
print("Trainer Knee Feet Ratio State 2: ", trainer_knee_feet_ratio_state2)

print("\nState 3:-\n")
print("Trainer Hip Angle State 3: ", trainer_hip_angle_state3)
print("Trainer Knee Angle State 3: ", trainer_knee_angle_state3)
print("Trainer Knee Feet Ratio State 3: ", trainer_knee_feet_ratio_state3)

State 1:-

Trainer Hip Angle State 1:  [178.4, 172.3, 166.0, 163.1, 162.5, 162.0, 161.4, 160.9, 159.0, 158.8, 158.0, 158.0, 158.1, 158.4, 158.7, 159.0, 159.2, 159.3, 159.5, 160.1, 161.2, 161.9, 163.6, 167.3, 172.8, 177.9, 173.2, 167.4, 163.8, 160.9, 160.6, 159.8, 157.4, 156.5, 156.8, 157.4, 157.7, 157.8, 157.2, 156.8, 156.8, 157.1, 157.6, 158.3, 159.3, 160.2, 163.5, 168.6, 173.4, 176.3, 169.9, 165.7, 163.9, 162.5, 162.0, 160.9, 159.5, 160.2, 161.0, 161.2, 161.1, 161.0, 161.0, 161.3, 161.6, 161.4, 162.1, 162.4, 163.2, 166.6, 171.8, 168.3, 174.9, 178.0, 170.5, 167.4, 164.0, 162.7, 162.1, 160.2, 158.7, 156.6, 154.7, 156.7, 157.2, 157.6, 156.5, 157.1, 155.5, 155.4]
Trainer Knee Angle State 1:  [160.3, 165.3, 169.4, 170.3, 170.6, 171.1, 171.3, 170.9, 171.9, 171.6, 171.9, 171.7, 171.8, 171.8, 171.7, 171.6, 171.4, 171.2, 170.9, 169.8, 169.1, 168.1, 166.5, 163.7, 160.7, 160.3, 166.9, 171.7, 174.4, 176.0, 176.3, 176.5, 177.3, 176.8, 176.0, 174.6, 173.6, 172.8, 172.6, 173.0, 173.1, 172.6, 171.9,

In [11]:
# print min and max angles for each joint

print("Min and max of angles:-\n")
print("\nState 1:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state1), max(trainer_hip_angle_state1))
print("Knee Angle: ", min(trainer_knee_angle_state1), max(trainer_knee_angle_state1))
print("Knee Feet Ratio: ", min(trainer_knee_feet_ratio_state1), max(trainer_knee_feet_ratio_state1))

print("\nState 2:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state2), max(trainer_hip_angle_state2))
print("Knee Angle: ", min(trainer_knee_angle_state2), max(trainer_knee_angle_state2))
print("Knee Feet Ratio: ", min(trainer_knee_feet_ratio_state2), max(trainer_knee_feet_ratio_state2))

print("\nState 3:-\n")
print("Hip Angle: ", min(trainer_hip_angle_state3), max(trainer_hip_angle_state3))
print("Knee Angle: ", min(trainer_knee_angle_state3), max(trainer_knee_angle_state3))
print("Knee Feet Ratio: ", min(trainer_knee_feet_ratio_state3), max(trainer_knee_feet_ratio_state3))

# put min max in variables

hip_angle_state1_min = min(trainer_hip_angle_state1)
hip_angle_state1_max = max(trainer_hip_angle_state1)

knee_angle_state1_min = min(trainer_knee_angle_state1)
knee_angle_state1_max = max(trainer_knee_angle_state1)

knee_feet_ratio_min_state1 = min(trainer_knee_feet_ratio_state1)
knee_feet_ratio_max_state1 = max(trainer_knee_feet_ratio_state1)


hip_angle_state2_min = min(trainer_hip_angle_state2)
hip_angle_state2_max = max(trainer_hip_angle_state2)

knee_angle_state2_min = min(trainer_knee_angle_state2)
knee_angle_state2_max = max(trainer_knee_angle_state2)

knee_feet_ratio_min_state2 = min(trainer_knee_feet_ratio_state2)
knee_feet_ratio_max_state2 = max(trainer_knee_feet_ratio_state2)


hip_angle_state3_min = min(trainer_hip_angle_state3)
hip_angle_state3_max = max(trainer_hip_angle_state3)

knee_angle_state3_min = min(trainer_knee_angle_state3)
knee_angle_state3_max = max(trainer_knee_angle_state3)

knee_feet_ratio_min_state3 = min(trainer_knee_feet_ratio_state3)
knee_feet_ratio_max_state3 = max(trainer_knee_feet_ratio_state3)

Min and max of angles:-


State 1:-

Hip Angle:  154.7 178.4
Knee Angle:  160.3 179.9
Knee Feet Ratio:  0.7082286590412107 1.0773134168725305

State 2:-

Hip Angle:  113.6 179.2
Knee Angle:  115.0 157.9
Knee Feet Ratio:  0.9560612094057754 1.1652730094747954

State 3:-

Hip Angle:  83.5 133.0
Knee Angle:  92.5 113.9
Knee Feet Ratio:  1.1113217452489226 1.2995921333672455


<h1>Comparing trainer skeleton to user and evaluating</h1>

In [39]:
# Variables

leniency = 5
knee_feet_leniency = 0.25
errors = []
error_bool = False
depth_bool = False
knee_angle = 0
shoulder_angle = 0
hip_angle = 0
knee_angle = 0
reps = 0


# Test Variables

client_incorrect = []
trainer_incorrect = []

In [40]:
cap = cv2.VideoCapture(client_footage)

state = 0
states_visited = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            error_bool = False
            
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
             
            print("Knee Angle: ", knee_angle, "State: ", state)
           
           # State Selection
           
            if knee_angle >= state1_angle_threshold[0] and knee_angle <= state1_angle_threshold[1]:
                state = 1
                states_visited.append(1)
                
                if len(states_visited) == 0:
                    states_visited.append(1)
                elif states_visited.__contains__(1) and states_visited.__contains__(2) and states_visited.__contains__(3):
                    states_visited = []
                    states_visited.append(1)
                    depth_bool = False
                    reps += 1
                    print("Reps: ", reps)
                elif states_visited.__contains__(1) and states_visited.__contains__(2):
                    states_visited = []
                    errors.append("State 3 not visited, hit more depth")
                    error_bool = True
                    depth_bool = True
                    state = 0
                    client_incorrect.append("Full range of motion not performed")
                    trainer_incorrect.append("Full range of motion not performed")
                    reps += 1
                    print("Reps: ", reps)
                    
                
            elif knee_angle >= state2_angle_threshold[0] and knee_angle <= state2_angle_threshold[1]:
                state = 2
                states_visited.append(2)
                
            elif knee_angle >= state3_angle_threshold[0] and knee_angle <= state3_angle_threshold[1]:
                state = 3
                states_visited.append(3)
            else:
                state = 0
                states_visited.append(0)
            
            
            # Knee Feet Ratio Calculation
            
            knee_distance = calculate_distance(left_knee, right_knee)
            feet_distance = calculate_distance(left_ankle, right_ankle)
            user_knee_feet_ratio = knee_distance/feet_distance
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            
            # Checking Angles according to state
            
            
            trainer_hip_min = 0
            trainer_hip_max = 0
            
            trainer_knee_min = 0
            trainer_knee_max = 0
            
            trainer_knee_feet_ratio_min = 0
            trainer_knee_feet_ratio_max = 0
            
            if state == 1:
                
                trainer_hip_min = hip_angle_state1_min
                trainer_hip_max = hip_angle_state1_max
                
                trainer_knee_min = knee_angle_state1_min
                trainer_knee_max = knee_angle_state1_max
                
                trainer_knee_feet_ratio_min = knee_feet_ratio_min_state1
                trainer_knee_feet_ratio_max = knee_feet_ratio_max_state1
                
            elif state == 2:
                
                trainer_hip_min = hip_angle_state2_min
                trainer_hip_max = hip_angle_state2_max
                
                trainer_knee_min = knee_angle_state2_min
                trainer_knee_max = knee_angle_state2_max
                
                trainer_knee_feet_ratio_min = knee_feet_ratio_min_state2
                trainer_knee_feet_ratio_max = knee_feet_ratio_max_state2
                
            elif state == 3:
                
                trainer_hip_min = hip_angle_state3_min
                trainer_hip_max = hip_angle_state3_max
                
                trainer_knee_min = knee_angle_state3_min
                trainer_knee_max = knee_angle_state3_max
                
                trainer_knee_feet_ratio_min = knee_feet_ratio_min_state3
                trainer_knee_feet_ratio_max = knee_feet_ratio_max_state3
                
            else:
                
                trainer_hip_min = 0
                trainer_hip_max = 0
                
                trainer_knee_min = 0
                trainer_knee_max = 0
                
                trainer_knee_feet_ratio_min = 0
                trainer_knee_feet_ratio_max = 0
                
            
            
            # Perform checking of errors when user is not in state 0
            
            if state != 0: 
                
                
                # Hip Angle Matching
                
                if hip_angle < (trainer_hip_min - leniency):
                    errors.append("Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min - leniency)
                elif hip_angle > (trainer_hip_max + leniency):
                    errors.append("Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max + leniency)


                # Knee Angle Matching
                
                if knee_angle < (trainer_knee_min - leniency):
                    errors.append("Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min - leniency)
                elif knee_angle > (trainer_knee_max + leniency):
                    errors.append("Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max + leniency)
                    
                    
                # Knee Feet Ratio Matching
                
                if user_knee_feet_ratio < (trainer_knee_feet_ratio_min - knee_feet_leniency):
                    errors.append("Knees too close to one another")
                    error_bool = True
                    client_incorrect.append(user_knee_feet_ratio)
                    trainer_incorrect.append(trainer_knee_feet_ratio_min - knee_feet_leniency)
                elif user_knee_feet_ratio > (trainer_knee_feet_ratio_max + knee_feet_leniency):
                    errors.append("Knees too open")
                    error_bool = True
                    client_incorrect.append(user_knee_feet_ratio)
                    trainer_incorrect.append(trainer_knee_feet_ratio_max + knee_feet_leniency)
                    
                
            
        except:
            pass 
        
        if depth_bool == True:
            cv2.putText(image, "Increase Depth", (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        else:
            cv2.putText(image, "Depth Perfect", (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
                    
        if error_bool == True:
            cv2.putText(image, "Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
            
        else:
            cv2.putText(image, "No Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
        
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

Knee Angle:  176.00040698397996 State:  0
Knee Angle:  175.8393383858039 State:  1
Knee Angle:  175.62152488865473 State:  1
Knee Angle:  175.2737319207662 State:  1
Knee Angle:  175.14815707604083 State:  1
Knee Angle:  175.28399765340342 State:  1
Knee Angle:  175.0087783909824 State:  1
Knee Angle:  175.07151430590437 State:  1
Knee Angle:  174.89350833438655 State:  1
Knee Angle:  174.55084616890662 State:  1
Knee Angle:  174.5824988077414 State:  1
Knee Angle:  173.93834696190885 State:  1
Knee Angle:  173.83565644000294 State:  1
Knee Angle:  172.53795124716257 State:  1
Knee Angle:  171.68678310050063 State:  1
Knee Angle:  171.01669763452705 State:  1
Knee Angle:  169.57536890295904 State:  1
Knee Angle:  167.08820463662747 State:  1
Knee Angle:  163.32210483074226 State:  1
Knee Angle:  160.3381349649722 State:  1
Knee Angle:  156.73917875866965 State:  1
Knee Angle:  153.55730796325778 State:  2
Knee Angle:  150.97273637704475 State:  2
Knee Angle:  146.64766273634797 State: 

In [41]:
for i in range(len(errors)):
    print(errors[i] + " => Client: " + str(client_incorrect[i]) + " vs Trainer: " + str(trainer_incorrect[i]))

Knees too close to one another => Client: 0.6911281169382378 vs Trainer: 0.7060612094057754
Knees too close to one another => Client: 0.7056843768296593 vs Trainer: 0.7060612094057754
Knees too close to one another => Client: 0.6964631447920103 vs Trainer: 0.7060612094057754
Knees too close to one another => Client: 0.6762402895426527 vs Trainer: 0.7060612094057754
Knees too close to one another => Client: 0.6703551102317508 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.6733137850323789 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.7034335833665715 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.6805969268738804 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.6739312633053176 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.6678231392305335 vs Trainer: 0.8613217452489226
Knees too close to one another => Client: 0.6581229518935877 vs Trainer: 0.86132

In [28]:
cap = cv2.VideoCapture(0)

state = 0
states_visited = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            error_bool = False
            
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            
            # Knee Angle Calculation
            
            
            if right_vis_sum >= left_vis_sum:
                knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            else:
                knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
             
           
           # State Selection
           
            if knee_angle >= state1_angle_threshold[0] and knee_angle <= state1_angle_threshold[1]:
                state = 1
                states_visited.append(1)
                
                if len(states_visited) == 0:
                    states_visited.append(1)
                elif states_visited.__contains__(1) and states_visited.__contains__(2) and states_visited.__contains__(3):
                    states_visited = []
                    states_visited.append(1)
                    depth_bool = False
                    reps += 1
                    print("Reps: ", reps)
                elif states_visited.__contains__(1) and states_visited.__contains__(2):
                    states_visited = []
                    errors.append("State 3 not visited, hit more depth")
                    error_bool = True
                    depth_bool = True
                    state = 0
                    client_incorrect.append("Full range of motion not performed")
                    trainer_incorrect.append("Full range of motion not performed")
                    reps += 1
                    print("Reps: ", reps)
                    
                
            elif knee_angle >= state2_angle_threshold[0] and knee_angle <= state2_angle_threshold[1]:
                state = 2
                states_visited.append(2)
                
            elif knee_angle >= state3_angle_threshold[0] and knee_angle <= state3_angle_threshold[1]:
                state = 3
                states_visited.append(3)
            else:
                state = 0
                states_visited.append(0)
            
                
                
            # Hip Angle Calculation
            
            if right_vis_sum >= left_vis_sum:
                hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            else:
                hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            
            # Checking Angles according to state
            
            
            trainer_hip_min = 0
            trainer_hip_max = 0
            
            trainer_knee_min = 0
            trainer_knee_max = 0
            
            if state == 1:
                
                trainer_hip_min = hip_angle_state1_min
                trainer_hip_max = hip_angle_state1_max
                
                trainer_knee_min = knee_angle_state1_min
                trainer_knee_max = knee_angle_state1_max

                
            elif state == 2:
                
                trainer_hip_min = hip_angle_state2_min
                trainer_hip_max = hip_angle_state2_max
                
                trainer_knee_min = knee_angle_state2_min
                trainer_knee_max = knee_angle_state2_max
                
            elif state == 3:
                
                trainer_hip_min = hip_angle_state3_min
                trainer_hip_max = hip_angle_state3_max
                
                trainer_knee_min = knee_angle_state3_min
                trainer_knee_max = knee_angle_state3_max
                
            else:
                
                trainer_hip_min = 0
                trainer_hip_max = 0
                
                trainer_knee_min = 0
                trainer_knee_max = 0
            
            
            # Perform checking of errors when user is not in state 0
            
            if state != 0: 
                
                
                # Hip Angle Matching
                
                if hip_angle < (trainer_hip_min - leniency):
                    errors.append("Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min - leniency)
                elif hip_angle > (trainer_hip_max + leniency):
                    errors.append("Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max + leniency)


                # Knee Angle Matching
                
                if knee_angle < (trainer_knee_min - leniency):
                    errors.append("Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min - leniency)
                elif knee_angle > (trainer_knee_max + leniency):
                    errors.append("Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max + leniency)
                
            
        except:
            pass 
        
        if depth_bool == True:
            cv2.putText(image, "Increase Depth", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        else:
            cv2.putText(image, "Depth Perfect", (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
                    
        if error_bool == True:
            cv2.putText(image, "Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
            
        else:
            cv2.putText(image, "No Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
        
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

Reps:  1
Reps:  2
Reps:  3
Reps:  4
Reps:  5
Reps:  6
Reps:  7
Reps:  8
Reps:  9
Reps:  10
Reps:  11
Reps:  12
Reps:  13
Reps:  14
